In [3]:
# ---------------------------------------------
# Land Registration Blockchain with User Input
# ---------------------------------------------
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any
import hashlib, json, time, uuid
from copy import deepcopy

# ---------------- Transaction ---------------- #
@dataclass
class Transaction:
    tx_id: str
    type: str   # "REGISTER" | "TRANSFER" | "GENESIS"
    payload: Dict[str, Any]
    timestamp: float = field(default_factory=lambda: time.time())

    def to_ordered_dict(self) -> Dict[str, Any]:
        return {
            "tx_id": self.tx_id,
            "type": self.type,
            "payload": {k: self.payload[k] for k in sorted(self.payload.keys())},
            "timestamp": self.timestamp,
        }

# ---------------- Block ---------------- #
@dataclass
class Block:
    index: int
    timestamp: float
    transactions: List[Transaction]
    previous_hash: str
    nonce: int = 0
    hash: Optional[str] = None

    def compute_hash(self) -> str:
        block_dict = {
            "index": self.index,
            "timestamp": self.timestamp,
            "transactions": [tx.to_ordered_dict() for tx in self.transactions],
            "previous_hash": self.previous_hash,
            "nonce": self.nonce,
        }
        block_string = json.dumps(block_dict, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

# ---------------- Blockchain ---------------- #
class Blockchain:
    def __init__(self, difficulty: int = 3):
        self.difficulty = difficulty
        self.chain: List[Block] = []
        self.pending_transactions: List[Transaction] = []
        self.parcels: Dict[str, str] = {}
        self._create_genesis_block()

    def _create_genesis_block(self) -> None:
        genesis_tx = Transaction(str(uuid.uuid4()), "GENESIS", {"note": "Genesis Block"})
        genesis_block = Block(0, time.time(), [genesis_tx], "0"*64)
        self._mine_block(genesis_block)
        self.chain.append(genesis_block)

    def last_block(self) -> Block:
        return self.chain[-1]

    def _valid_proof(self, block: Block) -> bool:
        return (block.hash or "").startswith("0" * self.difficulty)

    def _mine_block(self, block: Block) -> None:
        while True:
            block.hash = block.compute_hash()
            if self._valid_proof(block):
                break
            block.nonce += 1

    def create_register_tx(self, parcel_id: str, location: str, area: float, new_owner: str) -> Transaction:
        tx = Transaction(str(uuid.uuid4()), "REGISTER",
                         {"parcel_id": parcel_id, "location": location, "area": float(area), "new_owner": new_owner})
        self._validate_register_tx(tx, state_preview=True)
        self.pending_transactions.append(tx)
        return tx

    def create_transfer_tx(self, parcel_id: str, from_owner: str, to_owner: str) -> Transaction:
        tx = Transaction(str(uuid.uuid4()), "TRANSFER",
                         {"parcel_id": parcel_id, "from_owner": from_owner, "to_owner": to_owner})
        self._validate_transfer_tx(tx, state_preview=True)
        self.pending_transactions.append(tx)
        return tx

    def _validate_register_tx(self, tx: Transaction, state_preview: bool = False, state: Optional[Dict[str, str]] = None):
        pid, area, new_owner = tx.payload["parcel_id"], tx.payload["area"], tx.payload["new_owner"]
        assert pid and new_owner, "Parcel ID and owner required"
        assert area > 0, "Area must be > 0"
        current_state = state if state is not None else self._preview_state() if state_preview else self.parcels
        assert pid not in current_state, f"Parcel '{pid}' already exists"

    def _validate_transfer_tx(self, tx: Transaction, state_preview: bool = False, state: Optional[Dict[str, str]] = None):
        pid, from_owner, to_owner = tx.payload["parcel_id"], tx.payload["from_owner"], tx.payload["to_owner"]
        current_state = state if state is not None else self._preview_state() if state_preview else self.parcels
        assert pid in current_state, f"Parcel '{pid}' does not exist"
        assert current_state[pid] == from_owner, f"Invalid owner, current owner is '{current_state[pid]}'"
        assert from_owner != to_owner, "From and To owners must be different"

    def _preview_state(self) -> Dict[str, str]:
        state = deepcopy(self.parcels)
        for tx in self.pending_transactions:
            if tx.type == "REGISTER":
                state[tx.payload["parcel_id"]] = tx.payload["new_owner"]
            elif tx.type == "TRANSFER":
                state[tx.payload["parcel_id"]] = tx.payload["to_owner"]
        return state

    def mine_pending(self) -> Optional[Block]:
        if not self.pending_transactions:
            return None
        block = Block(len(self.chain), time.time(), self.pending_transactions.copy(), self.last_block().hash or "")
        self._mine_block(block)
        for tx in block.transactions:
            if tx.type == "REGISTER":
                self.parcels[tx.payload["parcel_id"]] = tx.payload["new_owner"]
            elif tx.type == "TRANSFER":
                self.parcels[tx.payload["parcel_id"]] = tx.payload["to_owner"]
        self.chain.append(block)
        self.pending_transactions.clear()
        return block

    def get_owner(self, parcel_id: str) -> Optional[str]:
        return self.parcels.get(parcel_id)

    def parcel_history(self, parcel_id: str) -> List[Transaction]:
        history = []
        for block in self.chain:
            for tx in block.transactions:
                if tx.payload.get("parcel_id") == parcel_id:
                    history.append(tx)
        return history

    def is_chain_valid(self) -> bool:
        for i, block in enumerate(self.chain):
            if block.hash != block.compute_hash():
                return False
            if i > 0 and block.previous_hash != self.chain[i-1].hash:
                return False
            if not self._valid_proof(block):
                return False
        return True

# ---------------- Interactive Menu ---------------- #
def run_menu():
    bc = Blockchain(difficulty=3)

    while True:
        print("\n=== Land Registration Blockchain ===")
        print("1. Register a new parcel")
        print("2. Transfer parcel ownership")
        print("3. Mine pending transactions")
        print("4. View parcel owner")
        print("5. View parcel history")
        print("6. Validate blockchain")
        print("7. Exit")

        choice = input("Enter choice: ").strip()

        try:
            if choice == "1":
                pid = input("Parcel ID: ")
                loc = input("Location: ")
                area = float(input("Area (sq ft): "))
                owner = input("Owner name: ")
                bc.create_register_tx(pid, loc, area, owner)
                print("✅ Parcel registered and added to pending transactions.")

            elif choice == "2":
                pid = input("Parcel ID: ")
                from_owner = input("From owner: ")
                to_owner = input("To owner: ")
                bc.create_transfer_tx(pid, from_owner, to_owner)
                print("✅ Transfer transaction added to pending transactions.")

            elif choice == "3":
                block = bc.mine_pending()
                if block:
                    print(f"✅ Block {block.index} mined with {len(block.transactions)} transactions.")
                else:
                    print("⚠️ No pending transactions to mine.")

            elif choice == "4":
                pid = input("Parcel ID: ")
                owner = bc.get_owner(pid)
                print(f"Owner of {pid}: {owner if owner else 'Not found'}")

            elif choice == "5":
                pid = input("Parcel ID: ")
                history = bc.parcel_history(pid)
                if not history:
                    print("No history found.")
                else:
                    for tx in history:
                        print(f"{tx.type} -> {tx.payload} at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tx.timestamp))}")

            elif choice == "6":
                print("Blockchain valid?", bc.is_chain_valid())

            elif choice == "7":
                print("Exiting...")
                break

            else:
                print("❌ Invalid choice, try again.")
        except Exception as e:
            print("⚠️ Error:", e)

if __name__ == "__main__":
    run_menu()



=== Land Registration Blockchain ===
1. Register a new parcel
2. Transfer parcel ownership
3. Mine pending transactions
4. View parcel owner
5. View parcel history
6. Validate blockchain
7. Exit


KeyboardInterrupt: Interrupted by user